In [1]:
# in this script, we will use thetao to produce the mixed layer depth
# mixed layer depth is calculated as the depth at which the potential temperature
# differs from the upper layer temperature by at least 0.2 K

# by: Ty Janoski
# edited: 06.01.20

In [2]:
# import statements
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar
import glob
import warnings

In [ ]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
          'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
# read in ocean potential temperature
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for mod in models:
        print(mod)
        maximum_list = []
        for j in range(8):
            print(j)
            start = j * 225
            end = start + 225
            if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
                 'MIROC-ESM']:
                # read in air temperature from last 50 years of piControl
                thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      use_cftime=True).thetao.isel(
                    time=slice(start,end)).load()
            elif mod in ['FGOALS-g2']:
                thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      decode_times=False).thetao.isel(
                    time=slice(start,end)).load()
            else:
                path = glob.glob('/dx07/tylerj/CMIP5_output/abrupt4xCO2/thetao_Omon_'+mod+'*.nc')[0]
                thetao = xr.open_dataset(path,use_cftime=True).thetao.isel(
                    time=slice(start,end))
            # grab the upper ocean layer
            top_layer = thetao.isel(lev=0)
            lev_4d = xr.broadcast(thetao.lev,thetao)[0].copy()
            for i in range(1,len(thetao[0,:,0,0])):
                loc = abs(top_layer -thetao[:,i,:,:])  < 0.2
                lev_4d[i,:,:,:] = lev_4d[i,:,:,:].where(xr.ufuncs.isfinite(lev_4d[i-1,:,:,:])).where(loc)
            lev_4d = lev_4d.where(xr.ufuncs.isfinite(thetao)).load()
            maximum = lev_4d.max(dim='lev').rename('mlotst').load()
            maximum_list.append(maximum)
            del maximum, lev_4d, thetao
        maximum_out = xr.concat(maximum_list, dim='time').load()
        maximum_out.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_4xCO2.nc')
        del maximum_list, maximum_out

In [ ]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
          'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
# read in ocean potential temperature
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for mod in models:
        print(mod)
        maximum_list = []
        for j in range(4):
            print(j)
            start = -600 + (j*150)
            end = start+150
            if mod in ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
                 'MIROC-ESM']:
                # read in air temperature from last 50 years of piControl
                thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      use_cftime=True).thetao.isel(
                    time=slice(start,end)).load()
            elif mod in ['FGOALS-g2']:
                thetao = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'_*.nc',
                                          parallel=True,combine='by_coords',
                                      decode_times=False).thetao.isel(
                    time=slice(start,end)).load()
            else:
                path = glob.glob('/dx07/tylerj/CMIP5_output/piControl/thetao_Omon_'+mod+'*.nc')[0]
                thetao = xr.open_dataset(path,use_cftime=True).thetao.isel(
                    time=slice(start,end))
            # grab the upper ocean layer
            top_layer = thetao.isel(lev=0)
            lev_4d = xr.broadcast(thetao.lev,thetao)[0].copy()
            for i in range(1,len(thetao[0,:,0,0])):
                loc = abs(top_layer -thetao[:,i,:,:])  < 0.2
                lev_4d[i,:,:,:] = lev_4d[i,:,:,:].where(xr.ufuncs.isfinite(lev_4d[i-1,:,:,:])).where(loc)
            lev_4d = lev_4d.where(xr.ufuncs.isfinite(thetao)).load()
            maximum = lev_4d.max(dim='lev').rename('mlotst').load()
            maximum_list.append(maximum)
            del lev_4d, maximum, thetao
        maximum_out = xr.concat(maximum_list, dim='time').load()
        maximum_out.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+mod+'_mlotst_pic.nc')
        del maximum_list, maximum_out